<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Mounted at /content/gdrive


In [4]:
from sklearn.preprocessing import LabelEncoder

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/images/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/images/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas

dftrain = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})
# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
dftrain['class_encoded'] = label_encoder.fit_transform(dftrain['class'])

# Convertir las etiquetas codificadas en un vector one-hot
class_one_hot = pd.get_dummies(dftrain['class_encoded'], prefix='class')

# Renombrar las columnas del vector one-hot
class_one_hot.columns = ['normal', 'benign', 'malignant']

# Concatenar el DataFrame original con las etiquetas one-hot
df = pd.concat([dftrain[['image_filename']], class_one_hot], axis=1)

# Imprimir las primeras filas del DataFrame para verificar
print(len(imgtrain_dir))

22


In [5]:
# Dividir el conjunto en entrenamiento, validación y test (70:15:15)
val_size = int(len(df) * 0.15)
test_size = int(len(df) * 0.15)

df = df.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = df[:test_size]
dfval = df[test_size:test_size+val_size]
dftrain = df[test_size+val_size:]

print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)
dftest = dftest.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 237
Número de ejemplos del conjunto de validación: 50
Número de ejemplos del conjunto de test: 50


In [6]:
import os
from PIL import Image

archivos = os.listdir(imgtrain_dir)
anchuras = []
alturas = []
for archivo in archivos:
      imagen = Image.open(os.path.join(imgtrain_dir, archivo))
      ancho, alto = imagen.size
      anchuras.append(ancho)
      alturas.append(alto)
media_anchura = sum(anchuras) / len(anchuras)
media_altura = sum(alturas) / len(alturas)

print("Media de anchura:", media_anchura)
print("Media de altura:", media_altura)

Media de anchura: 634.9139465875371
Media de altura: 514.9317507418398


In [31]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 634, 514
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = ['normal', 'benign', 'malignant']  # lista de nombres de las columnas de las etiquetas

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_filename, label_one_hot):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    # Redimensionar la imagen al tamaño deseado con relleno de ceros si es necesario
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0                               # normalización
    label = label_one_hot                                # Utilizar etiquetas codificadas como vector one-hot
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_filenames = df[x_col].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')


Número de lotes del conjunto de entrenamiento: 8
Número de lotes del conjunto de validación: 2
Número de lotes del conjunto de test: 2


In [32]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def f1_score(y_true, y_pred):
    """
    Calculate the F1 score.
    """
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_val

def get_model(img_width, img_height, num_classes):
    # Cargar la base convolucional del modelo VGG16 pre-entrenado en ImageNet
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Ajustar los parámetros de las nuevas capas del modelo, dejando fijos los parámetros del resto de capas
    for layer in base_model.layers:
        layer.trainable = False

    # Añadir nuevas capas a continuación de la base convolucional, para resolver la tarea de aprendizaje
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)

    # Añadir una última capa completamente conectada con el número de clases para obtener la salida de la red
    predictions = Dense(num_classes, activation='softmax')(x)

    # Crear el modelo final
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Create the model
model = get_model(img_width, img_height, n_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[f1_score])

# Print the model summary
model.summary()



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 634, 514, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 634, 514, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 634, 514, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 317, 257, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 317, 257, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 317, 257, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 158, 128, 128)     0   

In [33]:
# Entrenar el modelo con los datos preparados previamente
history = model.fit(train_dataset,
                    epochs=8,   # número de epochs
                    verbose=2,  # muestra información al finalizar cada epoch
                    validation_data=val_dataset)

# Imprimir el error mínimo de entrenamiento y validación
train_loss = np.array(history.history['loss'])
print(f'\nError mínimo en entrenamiento: {min(train_loss):.6f}')

val_loss = np.array(history.history['val_loss'])
print(f'Error mínimo en validación: {min(val_loss):.6f}')

# Imprimir el F1 score máximo de entrenamiento y validación
train_f1 = np.array(history.history['f1_score'])
print(f'\nF1 score máximo en entrenamiento: {max(train_f1):.6f}')

val_f1 = np.array(history.history['val_f1_score'])
print(f'F1 score máximo en validación: {max(val_f1):.6f}')

Epoch 1/8
8/8 - 64s - loss: 1.0131 - f1_score: 0.3775 - val_loss: 1.2190 - val_f1_score: 0.4410 - 64s/epoch - 8s/step
Epoch 2/8
8/8 - 8s - loss: 0.9992 - f1_score: 0.2804 - val_loss: 1.0359 - val_f1_score: 0.0000e+00 - 8s/epoch - 1s/step
Epoch 3/8
8/8 - 7s - loss: 0.9337 - f1_score: 0.4157 - val_loss: 1.0782 - val_f1_score: 0.4480 - 7s/epoch - 910ms/step
Epoch 4/8
8/8 - 7s - loss: 0.9289 - f1_score: 0.5052 - val_loss: 1.0413 - val_f1_score: 0.3861 - 7s/epoch - 894ms/step
Epoch 5/8
8/8 - 7s - loss: 0.8869 - f1_score: 0.5263 - val_loss: 1.0211 - val_f1_score: 0.4500 - 7s/epoch - 899ms/step
Epoch 6/8
8/8 - 8s - loss: 0.8874 - f1_score: 0.5359 - val_loss: 1.0085 - val_f1_score: 0.4682 - 8s/epoch - 1s/step
Epoch 7/8
8/8 - 8s - loss: 0.8592 - f1_score: 0.5761 - val_loss: 1.0057 - val_f1_score: 0.4844 - 8s/epoch - 1s/step
Epoch 8/8
8/8 - 7s - loss: 0.8513 - f1_score: 0.5733 - val_loss: 0.9839 - val_f1_score: 0.4643 - 7s/epoch - 927ms/step

Error mínimo en entrenamiento: 0.851299
Error mínimo 

In [34]:
# Evaluar el modelo en el conjunto de test
test_loss, test_acc = model.evaluate(test_dataset, verbose=1)
print("test_loss: %.4f, test_f1: %.4f" % (test_loss, test_acc))

2/2 [==============================] - 2s 563ms/step - loss: 0.8493 - f1_score: 0.5963
test_loss: 0.8493, test_f1: 0.5963


In [37]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input


# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame({'image_filename': test_filenames})

# Crear columnas para 'normal', 'benign' y 'malignant' con valores iniciales de 0
predictions_df['normal'] = 0
predictions_df['benign'] = 0
predictions_df['malignant'] = 0

# Umbral de probabilidad
threshold = 0.1

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)

    # Aplicar umbral de probabilidad y asignar clases
    if prediction[0][0] > threshold:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'normal'] = 1
    if prediction[0][1] > threshold:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'benign'] = 1
    if prediction[0][2] > threshold:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'malignant'] = 1

# Ordenar el DataFrame por el nombre del archivo de imagen
predictions_df = predictions_df.sort_values(by='image_filename')

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(predictions_df.head())


1/1 [==============================] - 0s 21ms/step
   image_filename  normal  benign  malignant
32  image_005.png       1       0          0
5   image_006.png       1       0          0
45  image_007.png       1       0          0
87  image_009.png       1       0          0
65  image_010.png       1       0          0
